## Assignment 1 (50 marks)
#### =====================================================================================================
### Deadline: 09/14 11:59 pm
#### =====================================================================================================

## Decision Tree Classification

### Problem 1 (25 marks)

`lab01_dataset_1.csv` contains a mixture of numerical and categorical data. Your task will be to write a function `my_ID3()` (without using the sklearn decision tree library) which can create a decision tree for the given dataset using the ID3 algorithm.

### 1.a (5 marks)

ID3 cannot handle continuous numerical data. Perform the necessary operations to convert the continuous-valued attribute to boolean attributes (refer to page 40 of Decision Tree.pdf). Do not use any of the sklearn library for this task. Display the updated dataset after handling continuous-valued attribute.

In [4]:
import pandas as pd
df = pd.read_csv("lab01_dataset_1.csv")
threshold = df["Score"].median() 
df["Score_high"] = df["Score"] > threshold
print("Threshold used for Score:", threshold)
print(df)


Threshold used for Score: 69.0
       Mood  Effort  Score Output  Score_high
0     Happy     Low     35    Yes       False
1     Happy  Medium     91     No        True
2     Happy    High     52     No       False
3   Neutral     Low     83     No        True
4   Neutral  Medium     48     No       False
5   Neutral    High     61     No       False
6       Sad     Low     40    Yes       False
7       Sad  Medium     98     No        True
8       Sad    High     73    Yes        True
9     Happy     Low     44    Yes       False
10    Happy  Medium     86     No        True
11    Happy    High     39    Yes       False
12    Happy     Low     66     No       False
13  Neutral  Medium     75    Yes        True
14  Neutral    High     50     No       False
15      Sad     Low     69     No       False
16      Sad  Medium     70    Yes        True
17      Sad    High     95     No        True
18      Sad     Low     80    Yes        True


### 1.b (6 marks)

Next, you will have to ensure that the newly obtained dataset is optimal and free of errors. Take appropriate actions based on the outcomes of the following steps:

Check if the dataset has any missing values. (2 marks)<br>
Check if the dataset has any redundant or repeated input sample. (2 marks)<br>
Check if the dataset has any contradicting <input, output> pairs. (2 marks)

Do note that you need to perform all the above steps programmatically and not simply by visual examination.

In [15]:
import pandas as pd

miss = df.isnull().sum()
has_miss = miss.sum() > 0
y = df.columns[-1]
X = [c for c in df.columns if c != y]

dupes = df.duplicated(subset=X, keep=False)
dupe_rows = df[dupes].sort_values(by=X)
grp = df.groupby(X)[y].nunique()
contr = grp[grp > 1]

out = {
    "thresh": threshold,
    "has_missing": has_miss,
    "missing_counts": miss.to_dict(),
    "num_dupes": int(dupes.sum()),
    "dupe_examples": dupe_rows.head(10).to_dict(orient="records"),
    "num_contr": int(contr.shape[0]),
    "contradictions": contr.to_dict()
}
print(out)

df_new = df.drop_duplicates(subset=X, keep="first")
bad = contr.index
if len(bad) > 0:
    mask = df_new[X].apply(tuple, axis=1).isin(bad)
    df_new = df_new[~mask]

print("\n=== Final Cleaned DataFrame ===")
df_new = df_new.drop(columns=["Score"]).copy()
print(df_new)


{'thresh': np.float64(69.0), 'has_missing': np.False_, 'missing_counts': {'Mood': 0, 'Effort': 0, 'Score': 0, 'Output': 0, 'Score_high': 0}, 'num_dupes': 0, 'dupe_examples': [], 'num_contr': 0, 'contradictions': {}}

=== Final Cleaned DataFrame ===
       Mood  Effort Output  Score_high
0     Happy     Low    Yes       False
1     Happy  Medium     No        True
2     Happy    High     No       False
3   Neutral     Low     No        True
4   Neutral  Medium     No       False
5   Neutral    High     No       False
6       Sad     Low    Yes       False
7       Sad  Medium     No        True
8       Sad    High    Yes        True
9     Happy     Low    Yes       False
10    Happy  Medium     No        True
11    Happy    High    Yes       False
12    Happy     Low     No       False
13  Neutral  Medium    Yes        True
14  Neutral    High     No       False
15      Sad     Low     No       False
16      Sad  Medium    Yes        True
17      Sad    High     No        True
18      Sa

### 1.c (10 marks)

Your function `my_ID3()` should operate in a manner such that after ever round of decision making, it will output the attributes and its associated gain, with a message stating “Attribute X with Gain = Y is chosen as the decision attribute”. You can obviously create other functions as needed for completing `my_ID3()`.

In [16]:
import numpy as np
import pandas as pd

def H(y: pd.Series) -> float:
    cnt = y.value_counts()
    p = cnt / cnt.sum()
    return float(-(p * np.log2(p + 1e-12)).sum())

def gain(df: pd.DataFrame, col: str, ycol: str) -> float:
    H0 = H(df[ycol])
    n = len(df)
    Hc = 0.0
    for _, sub in df.groupby(col):
        Hc += (len(sub) / n) * H(sub[ycol])
    return H0 - Hc

def majority(y: pd.Series):
    return y.value_counts().idxmax()

def my_ID3(data: pd.DataFrame):
    ycol = data.columns[-1]
    feats = [c for c in data.columns if c != ycol]

    if data[ycol].nunique() == 1:
        return data[ycol].iloc[0]
    if len(feats) == 0:
        return majority(data[ycol])

    gains = {f: gain(data, f, ycol) for f in feats}
    for f, g in sorted(gains.items(), key=lambda kv: kv[1], reverse=True):
        print(f"Gain({f}) = {g:.6f}")
    best = max(gains, key=gains.get)
    gbest = gains[best]
    print(f'Attribute "{best}" with Gain = {gbest:.6f} is chosen as the decision attribute')

    T = {best: {}}
    for val, sub in data.groupby(best):
        T[best][val] = my_ID3(sub.drop(columns=[best]))
    return T

tree = my_ID3(df_clean)
print(tree)


Gain(Effort) = 0.184751
Gain(Mood) = 0.106504
Gain(Output) = 0.001457
Attribute "Effort" with Gain = 0.184751 is chosen as the decision attribute
Gain(Mood) = 0.918296
Gain(Output) = 0.044110
Attribute "Mood" with Gain = 0.918296 is chosen as the decision attribute
Gain(Mood) = 0.469565
Gain(Output) = 0.005978
Attribute "Mood" with Gain = 0.469565 is chosen as the decision attribute
Gain(Output) = 0.251629
Attribute "Output" with Gain = 0.251629 is chosen as the decision attribute
Gain(Mood) = 0.316689
Gain(Output) = 0.109170
Attribute "Mood" with Gain = 0.316689 is chosen as the decision attribute
Gain(Output) = 1.000000
Attribute "Output" with Gain = 1.000000 is chosen as the decision attribute
{'Effort': {'High': {'Mood': {'Happy': np.False_, 'Neutral': np.False_, 'Sad': np.True_}}, 'Low': {'Mood': {'Happy': np.False_, 'Neutral': np.True_, 'Sad': {'Output': {'No': np.False_, 'Yes': np.False_}}}}, 'Medium': {'Mood': {'Happy': np.True_, 'Neutral': {'Output': {'No': np.False_, 'Yes': n

### 1.d (4 marks)

Display the decision tree. The representation of the decision tree is upto you. You can choose either a textual representation or a graphical one; either is fine.

In [17]:
def show_tree(T, tab=""):
    if not isinstance(T, dict):
        print(tab + f"-> {T}")
        return
    for a, br in T.items():
        for v, sub in br.items():
            print(tab + f"[{a} = {v}]")
            show_tree(sub, tab + "  ")

print("=== Decision Tree (Textual) ===")
show_tree(tree)


=== Decision Tree (Textual) ===
[Effort = High]
  [Mood = Happy]
    -> False
  [Mood = Neutral]
    -> False
  [Mood = Sad]
    -> True
[Effort = Low]
  [Mood = Happy]
    -> False
  [Mood = Neutral]
    -> True
  [Mood = Sad]
    [Output = No]
      -> False
    [Output = Yes]
      -> False
[Effort = Medium]
  [Mood = Happy]
    -> True
  [Mood = Neutral]
    [Output = No]
      -> False
    [Output = Yes]
      -> True
  [Mood = Sad]
    -> True


### Problem 2 (25 marks)

`lab01_dataset_2.csv` has a mixture of numerical and categorical data. For this problem, you will use sklearn's [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) for the classification task.

### 2.a (5 marks)

Encode all the categorial data into numerical data using `One-hot Encoding` and display the new dataset.

### 2.b (10 marks)

Using the encoded dataset, perform the supervised learning using [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). Employ the [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) of 80-20 during the training.

### 2.c (5 marks)

After the training is complete, show the results by predicting the class of the test set. Display the results of the prediction and test set side-by-side.

### 2.d (5 marks)

Display the decision tree; it can be either a textual representation or a graphical representation.